In [13]:
import os
import pandas as pd
import psycopg2
import re

In [14]:

try:
    # Remplace les valeurs par tes propres informations de connexion
    db = psycopg2.connect(
        dbname="recsys",
        user="lea",
        password="prudence2006",
        host="localhost",
        port="5432"
    )
    print("Connexion réussie !")
    cursor = db.cursor()
except Exception as e:
    print(f"Erreur lors de la connexion : {e}")
finally:
    if 'connection' in locals():
        connection.close()

Connexion réussie !


In [15]:
dossier_principal = os.path.abspath('articles')

In [16]:
dossier_principal = "articles"
if os.path.exists(dossier_principal):
    print(f"Le dossier existe : {dossier_principal}")
    for annee in os.listdir(dossier_principal):
        chemin_annee = os.path.join(dossier_principal, annee)

        if os.path.isdir(chemin_annee):  # Vérifier si c'est bien un dossier
            print(f"Dossier trouvé : {chemin_annee}")
else:
    print(f"Le dossier principal n'existe pas : {dossier_principal}")

Le dossier existe : articles
Dossier trouvé : articles/2014
Dossier trouvé : articles/2016
Dossier trouvé : articles/2017
Dossier trouvé : articles/2015


In [17]:

def insert_pays(cursor, nom):
    try:
        cursor.execute("INSERT INTO Pays (nom) VALUES (%s) ON CONFLICT (nom) DO NOTHING RETURNING id", (nom,))
        result = cursor.fetchone()
        if not result:  # Si déjà présent, récupérer l'id
            cursor.execute("SELECT id FROM Pays WHERE nom=%s", (nom,))
            result = cursor.fetchone()
        return result[0] if result else None
    except Exception as e:
        print(f"Erreur lors de l'insertion du pays : {e}")
        return None

In [18]:
def insert_ville(cursor, nom, id_pays):
    try:
        cursor.execute("""
            INSERT INTO Ville (nom, id_pays) 
            VALUES (%s, %s) ON CONFLICT (nom, id_pays) DO NOTHING RETURNING id
        """, (nom, id_pays))
        result = cursor.fetchone()
        if not result:
            cursor.execute("SELECT id FROM Ville WHERE nom=%s AND id_pays=%s", (nom, id_pays))
            result = cursor.fetchone()
        return result[0] if result else None
    except Exception as e:
        print(f"Erreur lors de l'insertion de la ville : {e}")
        return None

In [19]:
def insert_affiliation(cursor, nom, id_ville):
    try:
        cursor.execute("""
            INSERT INTO Affiliation (nom, id_ville) 
            VALUES (%s, %s) ON CONFLICT (nom, id_ville) DO NOTHING RETURNING id
        """, (nom, id_ville))
        result = cursor.fetchone()
        if not result:  # Si déjà présent, récupérer l'id
            cursor.execute("SELECT id FROM Affiliation WHERE nom=%s AND id_ville=%s", (nom, id_ville))
            result = cursor.fetchone()
        return result[0] if result else None
    except Exception as e:
        print(f"Erreur lors de l'insertion de l'affiliation : {e}")
        return None

In [20]:

def insert_auteur(cursor, nom, id_affiliation):
    try:
        cursor.execute("""
            INSERT INTO Auteurs (nom, id_affiliation) 
            VALUES (%s, %s) ON CONFLICT (nom, id_affiliation) DO NOTHING RETURNING id
        """, (nom, id_affiliation))
        result = cursor.fetchone()
        if not result:
            cursor.execute("""
                SELECT id FROM Auteurs 
                WHERE nom=%s AND id_affiliation=%s
            """, (nom, id_affiliation))
            result = cursor.fetchone()
        return result[0] if result else None
    except Exception as e:
        print(f"Erreur lors de l'insertion de l'auteur : {e}")
        return None

In [21]:

def insert_article(cursor, titre ,annee_publication):
    try:
        cursor.execute("""
            INSERT INTO Articles (titre,annee_publication , conference) 
            VALUES (%s,%s, 'RecSys') ON CONFLICT (titre , annee_publication) DO NOTHING RETURNING id
        """, (titre, annee_publication))
        result = cursor.fetchone()
        if not result:
            cursor.execute("SELECT id FROM Articles WHERE titre=%s AND annee_publication=%s", (titre, annee))
            result = cursor.fetchone()
        return result[0] if result else None
    except Exception as e:
        print(f"Erreur lors de l'insertion de l'article : {e}")
        return None

In [22]:

def insert_article_auteur(cursor, id_article, id_auteur):
    try:
        cursor.execute("""
            INSERT INTO Article_Auteur (id_article, id_auteur) 
            VALUES (%s, %s) ON CONFLICT DO NOTHING
        """, (id_article, id_auteur))
        db.commit()  # Valider la transaction
    except Exception as e:
        print(f"Erreur lors de l'insertion de l'article-auteur : {e}")


In [23]:
import os

dossier_principal = "articles"  # Exemple de chemin à adapter

# Vérifier si le dossier principal existe
if not os.path.exists(dossier_principal):
    print(f"Le dossier principal n'existe pas : {dossier_principal}")

for dossier in os.listdir(dossier_principal):
    dossier_path = os.path.join(dossier_principal, dossier)
    if os.path.isdir(dossier_path):
        print(f"Dossier trouvé : {dossier_path}")
        fichiers = os.listdir(dossier_path)
        print(f"Fichiers dans {dossier_path} : {fichiers}")  # Afficher les fichiers
    else:
        print(f"{dossier_path} n'est pas un dossier.")

Dossier trouvé : articles/2014
Fichiers dans articles/2014 : ['article10', 'article5', 'article6', 'article16', 'article19', 'article1', 'article15', 'article2', 'article17', 'article20', 'article4', 'article3', 'article18', 'article14', 'article9', 'article7', 'article13', 'article8', 'article12', 'article11']
Dossier trouvé : articles/2016
Fichiers dans articles/2016 : ['article25', 'article30', 'article10', 'article5', 'article33', 'article6', 'article31', 'article16', 'article19', 'article36', 'article1', 'article37', 'article22', 'article15', 'article27', 'article2', 'article21', 'article23', 'article17', 'article20', 'article4', 'article3', 'article18', 'article26', 'article14', 'article35', 'article28', 'article9', 'article32', 'article29', 'article24', 'article7', 'article13', 'article8', 'article12', 'article34', 'article11']
Dossier trouvé : articles/2017
Fichiers dans articles/2017 : ['article25', 'article10', 'article5', 'article6', 'article16', 'article19', 'article1', 'ar

In [24]:

# Parcourir les fichiers

for dossier in os.listdir(dossier_principal):
    dossier_path = os.path.join(dossier_principal, dossier)
    if os.path.isdir(dossier_path):
        annee_publication = int(dossier)
     
        for fichier in os.listdir(dossier_path):
            if fichier.endswith(""):
                fichier_path = os.path.join(dossier_path, fichier)
             
                try :
                    with open(fichier_path, "r") as f:
                        lignes = f.readlines()
                     
                        titre = lignes[0].strip()
                        auteurs = lignes[1].strip().split("|")

                        id_article = insert_article(cursor ,titre , annee_publication)
                        print(f"Insertion de l'article avec ID : {id_article}")
                     
                        for auteur in auteurs:
                            match = re.search(r"(.+?), (.+?), (.+?), (.+)", auteur.strip())
                            if match:
                                nom, institution, ville, pays = match.groups()
                                id_pays = insert_pays(cursor , pays.strip())
                                id_ville = insert_ville(cursor , ville.strip(), id_pays)
                                id_affiliation = insert_affiliation(cursor ,institution.strip(), id_ville)
                                id_auteur = insert_auteur(cursor , nom.strip(), id_affiliation)
                                insert_article_auteur(cursor , id_article, id_auteur)
                                print(f"Auteur ajouté : {nom}, ID : {id_auteur}")


                        db.commit()  # Valider les transactions
                except Exception as e:
                    print(f"Erreur lors de l'insertion : {e}")
                    db.rollback()  # Annuler les changements en cas d'erreur
               
cursor.close()  # Fermer le curseur
db.close()  # Fermer la connexion

Insertion de l'article avec ID : 1
Auteur ajouté : Yuta Saito, ID : 1
Insertion de l'article avec ID : 2
Auteur ajouté : Yin Zhang, ID : 2
Auteur ajouté : Ziwei Zhu, ID : 3
Auteur ajouté : Yun He, ID : 4
Auteur ajouté : James A Caverlee, ID : 5
Insertion de l'article avec ID : 3
Auteur ajouté : Casper Hansen, ID : 6
Auteur ajouté : Christian Hansen, ID : 7
Auteur ajouté : Lucas Maystre, ID : 8
Auteur ajouté : Rishabh Mehrotra, ID : 9
Insertion de l'article avec ID : 4
Auteur ajouté : Samarth Aggarwal, ID : 10
Auteur ajouté : Rohin Garg, ID : 11
Auteur ajouté : Abhilasha Sancheti, ID : 12
Auteur ajouté : Bhanu Prakash Guda, ID : 13
Auteur ajouté : Iftikhar Ahamath Burhanuddin, ID : 14
Insertion de l'article avec ID : 5
Auteur ajouté : Jin Huang, ID : 15
Auteur ajouté : Harrie Oosterhuis, ID : 16
Auteur ajouté : Maarten de Rijke, ID : 17
Auteur ajouté : Herke van Hoof, ID : 18
Insertion de l'article avec ID : 6
Auteur ajouté : Yoshifumi Seki, ID : 19
Auteur ajouté : Takanori Maehara, ID 

Auteur ajouté : Dong Wang, ID : 99
Insertion de l'article avec ID : 29
Auteur ajouté : Paul Covington, ID : 100
Auteur ajouté : Jay Adams, ID : 101
Auteur ajouté : Emre Sargin, ID : 102
Insertion de l'article avec ID : 30
Auteur ajouté : Mike Gartrell, ID : 103
Auteur ajouté : Ulrich Paquet, ID : 104
Auteur ajouté : Noam Koenigstein, ID : 105
Insertion de l'article avec ID : 31
Auteur ajouté : Dietmar Jannach, ID : 106
Auteur ajouté : Gediminas Adomavicius, ID : 107
Insertion de l'article avec ID : 32
Auteur ajouté : Chaoyuan Wu, ID : 108
Auteur ajouté : Christopher V Alvino, ID : 109
Auteur ajouté : Alexander Smola, ID : 110
Auteur ajouté : Justin Basilico, ID : 111
Insertion de l'article avec ID : 33
Auteur ajouté : Michael D. Ekstrand, ID : 112
Auteur ajouté : Martijn C. Willemsen, ID : 113
Insertion de l'article avec ID : 34
Auteur ajouté : Qingpeng Cai, ID : 114
Auteur ajouté : Aris Filos-Ratsikas, ID : 115
Auteur ajouté : Chang Liu, ID : 116
Auteur ajouté : Pingzhong Tang, ID : 1